<a href="https://colab.research.google.com/github/NuttakitDW/kbtg_data/blob/master/cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
cc = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/cc.csv')
dmg = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/demographics.csv')
kplus = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/kplus.csv')
train = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/test.csv')

In [32]:
cc.sort_values("cc_no", inplace = True)
cc.head()

,cc_no,pos_dt,cc_txn_amt
30686,2,2018-06-04,1000
30680,2,2018-03-10,800
30681,2,2018-05-14,800
30682,2,2018-05-07,800
30683,2,2018-06-11,1000


In [33]:
cc2 = cc.copy()
cc2 = cc2.drop('pos_dt', axis=1)
cc2.head()

,cc_no,cc_txn_amt
30686,2,1000
30680,2,800
30681,2,800
30682,2,800
30683,2,1000


In [34]:
cc2 = cc2.groupby(['cc_no'])['cc_txn_amt'].sum().reset_index()
cc2.head()

,cc_no,cc_txn_amt
0,2,26900
1,4,540000
2,5,12000
3,6,32000
4,7,16200


In [35]:
dmg2 = dmg.copy()
dmg2 = dmg2.merge(cc2, how='left', on='cc_no').fillna(0)
dmg2.head()

,id,cc_no,gender,ocp_cd,age,cc_txn_amt
0,1,1,2,9.0,5,0.0
1,1,98397,2,9.0,5,44700.0
2,2,2,2,3.0,4,26900.0
3,2,9740,2,3.0,4,38300.0
4,3,3,2,1.0,3,0.0


In [36]:
idc = dmg2.groupby(['id'])['cc_txn_amt'].sum().reset_index()
idc.head()

,id,cc_txn_amt
0,1,44700.0
1,2,65200.0
2,3,0.0
3,4,770000.0
4,5,12000.0


In [37]:
mdf = dmg.drop('cc_no', axis=1)
mdf = mdf.drop_duplicates()
mdf.head()

,id,gender,ocp_cd,age
0,1,2,9.0,5
2,2,2,3.0,4
4,3,2,1.0,3
5,4,2,3.0,5
7,5,2,9.0,4


In [38]:
main = mdf.copy()
main.tail()

,id,gender,ocp_cd,age
102404,64996,2,3.0,2
102405,64997,2,3.0,2
102406,64998,1,9.0,3
102407,64999,2,3.0,3
102408,65000,2,9.0,4


In [39]:
main['income'] = train['income']
main.head()

,id,gender,ocp_cd,age,income
0,1,2,9.0,5,20000.0
2,2,2,3.0,4,29000.0
4,3,2,1.0,3,18000.0
5,4,2,3.0,5,19000.0
7,5,2,9.0,4,16000.0


In [41]:
kcount = kplus.groupby(['id'])['kp_txn_count'].sum().reset_index()
kamt = kplus.groupby(['id'])['kp_txn_amt'].sum().reset_index()
kamt.tail()

,id,kp_txn_amt
49264,64995,960500
49265,64996,41200
49266,64997,201200
49267,64998,162000
49268,64999,494800


In [42]:
main = main.merge(kcount, how='left', on='id').fillna(0)
main = main.merge(kamt, how='left', on='id').fillna(0)
main.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt
0,1,2,9.0,5,20000.0,2.0,3500.0
1,2,2,3.0,4,29000.0,104.0,210000.0
2,3,2,1.0,3,18000.0,70.0,139700.0
3,4,2,3.0,5,19000.0,315.0,6076100.0
4,5,2,9.0,4,16000.0,0.0,0.0


In [43]:
main['cd_ex'] = idc['cc_txn_amt']
main.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex
0,1,2,9.0,5,20000.0,2.0,3500.0,44700.0
1,2,2,3.0,4,29000.0,104.0,210000.0,65200.0
2,3,2,1.0,3,18000.0,70.0,139700.0,0.0
3,4,2,3.0,5,19000.0,315.0,6076100.0,770000.0
4,5,2,9.0,4,16000.0,0.0,0.0,12000.0


In [44]:
main.isna().any()

id              False
gender          False
ocp_cd          False
age             False
income          False
kp_txn_count    False
kp_txn_amt      False
cd_ex           False
dtype: bool

In [45]:
train_data = main[:50000]
test_data = main[50000:]
test_data = test_data.drop('income', axis=1)
test_data.head()

,id,gender,ocp_cd,age,kp_txn_count,kp_txn_amt,cd_ex
50000,50001,1,9.0,3,28.0,64600.0,31800.0
50001,50002,1,9.0,4,131.0,93700.0,5900.0
50002,50003,1,3.0,4,83.0,202500.0,0.0
50003,50004,2,3.0,4,20.0,11000.0,7200.0
50004,50005,1,3.0,4,67.0,195500.0,14800.0


In [26]:
main.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex
0,1,2,9.0,5,20000.0,2.0,1750.000000,4470.000000
1,2,2,3.0,4,29000.0,104.0,8400.000000,2244.485294
2,3,2,1.0,3,18000.0,70.0,6652.380952,0.000000
3,4,2,3.0,5,19000.0,315.0,243044.000000,20585.284281
4,5,2,9.0,4,16000.0,0.0,0.000000,4000.000000


In [0]:
from google.colab import files

main.to_csv('main.csv', index=False)
files.download('main.csv')